# RecordLinkage Linking

> Leveraging the RL library to determine approximate matching over a range of fields using various methods.

In [ ]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx

#not currently using jellyfish
import jellyfish as jf

In [ ]:
outputDir = '..//scrapedData//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

In [ ]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
dfs = []
for key in sourceFiles.keys():
    vars()[key] = fx.mergeSourceFiles (key, outputDir, sourceFiles)
    dfs.append(vars()[key])

In [ ]:
allAmerican

In [ ]:
fx.createNewID(idConfig['sports247'], sports247, field_agg)
fx.createNewID(idConfig['rivals'], rivals, field_agg)
#createNewID(idConfig['cfbRef'], cfbRef, field_agg)
fx.createNewID(idConfig['allConf'], allConf, field_agg)
fx.createNewID(idConfig['allAmerican'], allAmerican, field_agg)
fx.createNewID(idConfig['nflData'], nflData, field_agg)
fx.createNewID(idConfig['ncaa'], ncaa, field_agg, ifAlternate=True)

In [ ]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)
#df_cfbRef = pandas.DataFrame(cfbRef)
df_allConf = pandas.DataFrame(allConf)
df_allAmerican = pandas.DataFrame(allAmerican)
df_nflData = pandas.DataFrame(nflData)
df_ncaa = pandas.DataFrame(ncaa)

In [ ]:
df_rivals.head()

In [ ]:
df_sports247.index.name = '247_index'
df_rivals.index.name = 'rivals_index'
indexer = recordlinkage.Index()
indexer.block('school')
indexer.block('year')
candidate_links = indexer.index(df_sports247, df_rivals)

In [ ]:
c = recordlinkage.Compare()

c.exact('ID', 'ID', label='ID')

c.string('playerName', 'playerName', method='damerau_levenshtein', label='playerName')
c.string('school', 'school', label='college')
c.string('city', 'city', label='city')
c.exact('state', 'state', label='state')
c.string('highSchool', 'highSchool', label='highSchool')
c.exact('position', 'position', label='position')
c.exact('year', 'year', label='year')

# The comparison vectors
features = c.compute(candidate_links, df_sports247, df_rivals)

features

In [ ]:
features.to_pickle("features.pkl")

In [ ]:
features = pandas.read_pickle("features.pkl")

In [ ]:
features['sum'] = (features['playerName'] + features['college'] + features['city'] + features['state'] + features['highSchool'] + features['year'] + features['position'])/7
features.describe()

In [ ]:
features.boxplot(column=['playerName', 'college', 'city', 'state', 'highSchool', 'position', 'year'])

In [ ]:
not_id = features[features['ID'] != 1]
not_id = (not_id.loc[not_id['sum'] > .945])
#not_id = (not_id.loc[not_id['sum'] > 6.62])

#all_matches = ((features.loc[features['sum'] > 6.62] & features.loc[features['ID'] == 0]) | (features.loc[features['ID'] == 1]))

In [ ]:
not_id.reset_index(inplace=True)

In [ ]:
noIDDict = not_id.to_dict('records')

In [ ]:
## Hey Sober Andrew - this all works now, but you have to clean-up the output
## cause there are dueling column names between the two collections.  
## Look at the summarize code to figure out how to handle that

#Can only include the reset_index the first time
df_sports247.reset_index(inplace=True)
df_rivals.reset_index(inplace=True)

sportDict = df_sports247.to_dict('records')
rivalsDict = df_rivals.to_dict('records')

finalSet = []
#print(df_sports247.to_dict('records'))
for record in noIDDict:
    finalRecord = {}
    for recruit in sportDict:
        #print(recruit)
        if (recruit['247_index'] == record['247_index']):
            for key,value in recruit.items():
                finalRecord[key] = value
            break
    for recruit in rivalsDict:
        #print(recruit)
        if (recruit['rivals_index'] == record['rivals_index']):
            for key,value in recruit.items():
                finalRecord[key] = value
            break
    finalSet.append(finalRecord)
print(finalSet)

In [ ]:
#df_sports247.head()
df_sports247.iloc[5618, :]

In [ ]:
test = df_rivals.iloc[2673, :]
test['year']

In [ ]:
features.iloc[2673, :]

In [ ]:
print(fuzzy_matches)

In [ ]:
ids_to_match = []
ids = fuzzy_matches['ID']
for key in ids.keys():
    i = {}
    print(key[0])
    i['247'] = key[0]
    i['rivals'] = key[1]
    ids_to_match.append(i)


In [ ]:
fuzzy_list = []
for i in ids_to_match:
    sports247_dict = pandas.DataFrame(df_sports247.iloc[i['247'], :]).to_dict()
    
    rivals_dict= pandas.DataFrame(df_rivals.iloc[i['rivals'], :]).to_dict()
    for i in rivals_dict:   
        sports247_dict.update(i)
    fuzzy_list.append(sport247_dict)

In [ ]:
print(fuzzy_list)

In [ ]:
df_rivals